<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

Наш проект включает в себя несколько этапов:
* знакомство с данными;
* предварительный анализ данных;
* детальный анализ вакансий;
* анализ работодателей;
* предметный анализ.

Требования к оформлению ноутбука-решения:

* Решение оформляется только в Jupyter Notebook.
* Решение оформляется в соответствии с ноутбуком-шаблоном.
* Каждое задание выполняется в отдельной ячейке, выделенной под задание (в шаблоне они помечены как ваш код здесь). Не следует создавать много ячеек для решения задачи — это провоцирует неудобства при проверке.
* Текст SQL-запросов и код на Python должны быть читаемыми. Не забывайте про отступы в SQL-коде.
* Выводы по каждому этапу оформляются в формате Markdown в отдельной ячейке (в шаблоне они помечены как ваши выводы здесь).
* Выводы можно дополнительно проиллюстрировать с помощью графиков. Они оформляются в соответствии с теми правилами, которые мы приводили в модуле по визуализации данных.
* Не забудьте удалить ячейку с данными соединения перед фиксацией работы в GitHub.

In [1]:
import pandas as pd
import psycopg2
import urllib.parse as req

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = '-'
USER = '-'
PASSWORD = '-'
HOST = '-'
PORT = '-'

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [71]:
# текст запроса
vac_count = f'''select
-- выводим общее количество строк 
    count(*) AS count_vacancies
-- обращаемся к таблице вакансий
from vacancies'''

In [72]:
# результат запроса
vac_count_df = pd.read_sql_query(vac_count, connection)
vac_count_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\1907099108.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vac_count_df = pd.read_sql_query(vac_count, connection)


,count_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [73]:
# текст запроса
emp_count = f'''select
-- выводим общее количество строк 
    count(*) AS count_employers
-- обращаемся к таблице работодателей
from employers'''

In [74]:
# результат запроса
emp_count_df = pd.read_sql_query(emp_count, connection)
emp_count_df


C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\1154262183.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emp_count_df = pd.read_sql_query(emp_count, connection)


,count_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [79]:
# текст запроса
ar_count = f'''select
-- выводим общее количество строк
    count(*) AS count_areas
-- обращаемся к таблице регионов 
from areas'''

In [80]:
# результат запроса
ar_count_df = pd.read_sql_query(ar_count, connection)
ar_count_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\3006970182.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ar_count_df = pd.read_sql_query(ar_count, connection)


,count_areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [77]:
# текст запроса
ind_count = f'''select
-- выводим общее количество строк
    count(*) AS count_industries
-- обращаемся к таблице со сферами деятельности
from industries'''

In [78]:
# результат запроса
ind_count_df = pd.read_sql_query(ind_count, connection)
ind_count_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\750610291.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ind_count_df = pd.read_sql_query(ind_count, connection)


,count_industries
0,294


***

# Выводы по предварительному анализу данных

Первичное изучение данных необходимо для получения понимания об объекте анализа и дальнейшей работы с ним. Предлагаемые к изучению таблицы имеют разную размерность, а также обладают ключами, позволяющими объединить данные, содержащиеся в этих таблицах в единое целое для более детального и информативного изучения.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [67]:
# текст запроса
count_areas = f'''select
-- выводим общее количество вакансий
-- и наименования регионов 
    count(v.name) AS cnt,
    a.name AS area
-- обращаемся к таблице вакансий    
from 
    vacancies AS v
-- объединяем таблицу вакансий с таблицей регионов
-- где id региона равен id региона размещения вакансии    
join areas AS a ON v.area_id = a.id
-- группируем по регионам
group by area
-- сортируем количество вакансий по убыванию 
order by count(v.name) desc'''

In [68]:
# результат запроса
count_areas_df = pd.read_sql_query(count_areas, connection)
count_areas_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\282430918.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_areas_df = pd.read_sql_query(count_areas, connection)


,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
...,...,...
764,1,Тарко-Сале
765,1,Новоаннинский
766,1,Бирск
767,1,Сасово


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [69]:
# текст запроса
count_salary_from_to = f'''select
-- выводим количество вакансий
    count(name) AS count_vac_with_salary
-- обращаемся к таблице вакансий    
from vacancies
-- фильтруем по наличию не пустых строк в столбцах 
-- с минимальной и максимальной зарплатой
where salary_from is not null or salary_to is not null'''

In [70]:
# результат запроса
count_salary_from_to_df = pd.read_sql_query(count_salary_from_to, connection)
count_salary_from_to_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\4016844712.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_salary_from_to_df = pd.read_sql_query(count_salary_from_to, connection)


,count_vac_with_salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [81]:
# текст запроса
avg_salary_from_to =  f'''select
-- выводим среднее значение 
-- нижнего и верхнего значений зарплатной вилки
    avg(salary_from) as salary_from,
    avg(salary_to) as salary_to
-- обращаемся к таблице вакансий
from vacancies'''    

In [82]:
# результат запроса
avg_salary_from_to_df = pd.read_sql_query(avg_salary_from_to,connection)
round(avg_salary_from_to_df)

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\913962371.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary_from_to_df = pd.read_sql_query(avg_salary_from_to,connection)


,salary_from,salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [83]:
# текст запроса
kombinasyon = f'''select
-- выводим количество вакансий,
-- тип рабочего графика,
-- тип трудоустройства  
    count(name) AS count_vacancies,
    schedule,
    employment 
-- обращаемся к таблице вакансий
from vacancies
-- группируем по типу рабочего графика
-- и типу трудоустройства
group by schedule, employment
-- сортируем вакансии по убыванию
order by count(name) desc'''

In [84]:
# результат запроса
kombinasyon_df = pd.read_sql_query(kombinasyon, connection)
kombinasyon_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\2252412564.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  kombinasyon_df = pd.read_sql_query(kombinasyon, connection)


,count_vacancies,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [85]:
# текст запроса
experience = f'''select
-- выводим требуемый опыт работы
-- и количество вакансий
    experience,
    count(name) AS vacancies
-- обращаемся к таблице вакансий
from vacancies
-- группируем по опыту работы
group by experience
-- сортируем вакансии по убыванию
order by count(name)''' 

In [86]:
# результат запроса
experience_df = pd.read_sql_query(experience, connection)
experience_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\38369888.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  experience_df = pd.read_sql_query(experience, connection)


,experience,vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

# Выводы по детальному анализу вакансий

Анализу были подвергнуты данные отражающие информацию о количестве вакансий, их месту нахождения и графиках работы, а также данные об необходимом опыте работы и о предлагаемой заработной плате. Из проведённого анализа следует, что максимальное количество вакансий сосредоточенно в пятерке городов:
- Москва - 5 333 вакансии;
- Санкт-Петербург - 2 851 вакансия;
- Минск - 2 112 вакансий;
- Новосибирск - 2 006 вакансий;
- Алматы - 1 892 вакансии.
Общее количество вакансий, приходящихся на эти города - 14 194, что является примерно третью от общего количества вакансий (49 197).

Стоит отметить, что более 50 % предлагаемых вакансий (26 152) требуют от соискателя наличие опыта работы от 1 года до 3 лет. Можно сделать вывод, что в целом, рынок труда нацелен на поиск начинающих специалистов, а также специалистов среднего уровня. Также большинство вакансий (35 367) в качестве предлагаемых типа рабочего графика и типа трудоустройства предоставляют соискателю тип графика - полный день, а тип трудоустройства - полная занятость. На втором по количеству вакансий месте - сочетание типов удаленная работа и полная занятость (7 902 вакансии).

Анализу также подверглась информативность вакансий, в частности было проанализировано наличие в вакансиях указаний на заработную плату. Так, примерно половина вакансий (24 073) содержит указание о заработной плате(проверялось наличие нижнего или верхнего порогов зарплатной вилки). Стоит обратить внимание, что в целом рынок труда предлагает довольно приемлемую оплату труда, поскольку средний показатель нижней границы зарплатной вилки составляет 71 065 рублей, а верхней границы - 110 537 рублей.



# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [87]:
# текст запроса
top_employers = f'''select
-- выводим наименование работодателя
-- и количество вакансий
    emp.name AS employers,
    count(v.name) AS count_vacancies
-- обращаемся к таблице работодателей
from employers AS emp
-- объединяем с таблицей вакансий
-- где id работодателя равно id работодателя в таблице вакансий
join vacancies AS v ON v.employer_id = emp.id
-- группируем по работодателям
group by employers
-- сортируем по вакансиям в порядке убывания
order by count(v.name) desc
-- выводим пять первых строк
limit 5''' 

In [88]:
# результат запроса
top_employers_df = pd.read_sql_query(top_employers, connection)
top_employers_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\2503381715.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_employers_df = pd.read_sql_query(top_employers, connection)


,employers,count_vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [89]:
# текст запроса
count_area_employers = f''' select
-- выводим наименование региона,
-- количество работодателей
-- и количество вакансий
    ar.name AS area,
    count(emp.name) AS count_employers,
    count(v.name) AS count_vacancies
-- обращаемся к таблице регионов
from areas AS ar
-- присоединяем ко всей таблице регионов таблицу работодателей
-- где id региона равен id региону работодателя
left join employers AS emp ON ar.id = emp.area
-- присоединяем таблицу вакансий
-- где id региона равен id региону вакансии
left join vacancies AS v ON ar.id = v.area_id
-- группируем по регионам
group by ar.name
-- фильтруем по количеству вакансий
having count(v.name) = 0
-- сортируем по количеству работодателей
order by count(emp.name) desc'''

In [90]:
# результат запроса
count_area_employers_df = pd.read_sql_query(count_area_employers, connection)
count_area_employers_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\1990352973.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_area_employers_df = pd.read_sql_query(count_area_employers, connection)


,area,count_employers,count_vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
588,Струнино,0,0
589,Славгород (Алтайский край),0,0
590,Сорочинск,0,0
591,Ленинский (Тульская область),0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [91]:
# текст запроса
employers_area = f'''select
-- выводим работодателей
-- и количество регионов 
    emp.name AS employer_name,
    count(distinct v.area_id) AS count_areas 
-- обращаемся к таблице работодателей
from employers AS emp
-- присоединяем таблицу вакансий
-- где id работодателя равен id вакансии работодателя
join vacancies AS v ON v.employer_id=emp.id
-- группируем по работодателю
group by emp.name
-- сортируем по количеству вакансий в порядке убывания
order by count(distinct v.area_id) desc'''

In [92]:
# результат запроса
employers_area_df = pd.read_sql_query(employers_area, connection)
employers_area_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\1420398628.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_area_df = pd.read_sql_query(employers_area, connection)


,employer_name,count_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [93]:
# текст запроса
emp_industries = f'''select 
-- выводим количество работодателей
    count(emp.name) AS count_emp_without_ind  
-- обращаемся к таблице работодателей
from employers AS emp
-- присоединяем ко всей таблице работодателей таблицу со сферами деятельности работодателей
-- где id работодателя равен id сфере деятельности работодателя
left join employers_industries AS ei ON emp.id = ei.employer_id
-- фильтруем по наличию пустых значений в столбце со сферой деятельности
where ei.industry_id is null'''

In [94]:
# результат запроса
emp_industries_df = pd.read_sql_query(emp_industries, connection)
emp_industries_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\3784121009.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emp_industries_df = pd.read_sql_query(emp_industries, connection)


,count_emp_without_ind
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [95]:
# текст запроса
emp_name = f'''select
-- выводим наименование работодателя 
    emp.name AS employer_name 
-- обращаемся к таблице работодателей
from employers AS emp
-- присоединяем ко всей таблице работодателей таблицу со сферами деятельности работодателей 
-- где id работодателя равен id сфере деятельности работодателя
left join employers_industries AS ei ON emp.id = ei.employer_id
-- группируем по id работодателя
group by emp.id 
-- фильтруем по количеству сфер деятельности
having count(ei.industry_id) = 4
-- сортируем по наименованию работодателя
order by emp.name
-- выводим третью строку
offset 2 limit 1'''

In [96]:
# результат запроса
emp_name_df = pd.read_sql_query(emp_name, connection)
emp_name_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\1393481143.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emp_name_df = pd.read_sql_query(emp_name, connection)


,employer_name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [97]:
# текст запроса
emp_ind = f'''select
-- выводим количество работодателей 
    count(emp.name) AS count_employers 
-- обращаемся к таблице работодателей
from employers AS emp
-- присоединяем ко всей таблице работодателей таблицу со сферами деятельности работодателей 
-- где id работодателя равен id сфере деятельности работодателя
left join employers_industries AS ei ON emp.id = ei.employer_id
-- присоединяем таблицу с вариантами деятельности работодателей
-- где id сферы деятельности работодателя равен id варианта сферы деятельности
left join industries AS ind ON ei.industry_id = ind.id 
-- фильтруем по названию сферы деятельности
where ind.name = 'Разработка программного обеспечения' '''

In [98]:
# результат запроса
emp_ind_df = pd.read_sql_query(emp_ind,connection)
emp_ind_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\3515320719.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emp_ind_df = pd.read_sql_query(emp_ind,connection)


,count_employers
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [99]:
# код для получения списка городов-милионников
# разбирает URL-адрес на составляющие
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
# собирает URL-адрес обратно с использованием метода экранирования 
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
# выделяем города
cities = pd.read_html(url)[1]
# преобразуем переменную с городами в кортеж
mil_cities = tuple(cities['Город'])
# выводим кортеж
display(mil_cities)


('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [100]:
# текст запроса
# формируем вспомогательный запрос
ya_areas= f'''with city as (
    select
-- выводим регионы
-- и количество вакансий     
        ar.name AS area,
        count(v.name) AS count_vacancies
-- обращаемся к таблице работодателей    
    from employers AS emp
-- присоединяем таблицу вакансий
-- где id вакансии работодателя равен id работодателя    
    join vacancies AS v ON v.employer_id = emp.id
-- присоединяем таблицу регионов
-- где id региона равен id региона вакансии    
    join areas AS ar ON ar.id = v.area_id
-- фильтруем по наименованию работодателя    
    where emp.name = 'Яндекс'
-- группируем по названию региона    
    group by ar.name
-- фильтруем по названию региона
-- где регионы это регионы из ранее созданного кортежа    
    having ar.name in {mil_cities}
-- сортируем по количеству вакансий в порядке убывания    
    order by count(v.name) desc   
)
-- основной запрос
    select
-- выводим название региона
-- и количество вакансий     
        city.area,
        city.count_vacancies
-- обращаемся к вспомогательному запросу
from city
union all
-- присоединяем строку total
-- и сумму  количества вакансий в регионах
select
    'total',
    sum(city.count_vacancies)
from city'''

In [101]:
# результат запроса
ya_areas_df = pd.read_sql_query(ya_areas, connection)
ya_areas_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\804695278.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ya_areas_df = pd.read_sql_query(ya_areas, connection)


,area,count_vacancies
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Нижний Новгород,36.0
4,Новосибирск,35.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Самара,26.0
8,Уфа,26.0
9,Казань,25.0


***

# Выводы по анализу работодателей

Анализу были подвергнуты данные отражающие информацию о работодателях, их сферах деятельности, а также регионах, в которых работодатели публикуют свои вакансии.

В топ 5 работодателей входят:
- Яндекс - 1 933 вакансии;
- Ростелеком - 491 вакансия;
- Тинькофф - 444 вакансии;
- СБЕР - 428 вакансий;
- Газпром нефть - 331 вакансия.

Стоит отметить, что компании Яндекс и Ростелеком также входят в топ пять компаний по количеству регионов в которых они размещают свои вакансии. При этом Яндекс сохраняет лидерство - 181 регионов, а Ростелеком с третьей строчки топа по количеству вакансий поднялся на вторую строчку топа по количеству регионов размещения вакансий - 152 региона.

Что касается сфер деятельности работодателей, стоит отметить, что 8 419 работодателей не указали сферу своей деятельности из чего можно сделать вывод, что такие работодатели менее интересны потенциальным соискателем, поскольку при поиске работы соискатели не только сопоставляют свои навыки и опыт работы с описанными в предлагаемой вакансии работодателя, но и подбирают себе работу исходя из собственных интересов к определенной (определенным) сфере деятельности и делают выбор в пользу работодателя разделяющего интересы соискателя.



# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [102]:
# текст запроса
data_vac = f'''select
-- выводим количество вакансий
    count(name) AS count_vacancies
-- обращаемся к таблице вакансий
from vacancies
-- фильтруем по наличию в названии вакансий ключевых слов
-- названия вакансий приведены в нижний регистр
where lower(name) like '%data%' 
    or lower(name) like '%данн%' '''

In [103]:
# результат запроса
data_vac_df = pd.read_sql_query(data_vac, connection)
data_vac_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\4138266289.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_vac_df = pd.read_sql_query(data_vac, connection)


,count_vacancies
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [106]:
# текст запроса
jds_vac = f'''select
-- выводим количество вакансий
    count(name) AS count_junior_ds_vac
-- обращаемся к таблице вакансий
from vacancies
-- фильтруем по ключевым словам в вакансиях
-- и по ключемым словам в опыте работы и типе занятости
where (name ilike '%data scientist%' 
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name ilike '%ML%'
    and name not ilike '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and 
    (name ilike '%junior%'
    or experience = 'Нет опыта'
    or employment = 'Стажировка')'''

In [107]:
# результат запроса
jds_vac_df = pd.read_sql_query(jds_vac, connection)
jds_vac_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\383992579.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  jds_vac_df = pd.read_sql_query(jds_vac, connection)


,count_junior_ds_vac
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [108]:
# текст запроса
sql_ds_vac = f'''select
-- выводим количество вакансий
    count(name) AS count_ds_vac_with_sql_skills
-- обращаемся к таблице вакансий
from vacancies
-- фильтруем по ключевым словам в вакансиях
-- и по ключевым словам в навыках
where (name ilike '%data scientist%' 
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name ilike '%ML%'
    and name not ilike '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and (key_skills ilike '%SQL%'
    or key_skills ilike '%postgres%')'''

In [109]:
# результат запроса
sql_ds_vac_df = pd.read_sql_query(sql_ds_vac, connection)
sql_ds_vac_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\4166581444.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_ds_vac_df = pd.read_sql_query(sql_ds_vac, connection)


,count_ds_vac_with_sql_skills
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [110]:
# текст запроса
python_ds_vac = f'''select
-- выводим количество вакансий
    count(name) count_ds_vac_with_py_skills
-- обращаемся к таблице вакансий
from vacancies
-- фильтруем по ключевым словам в вакансиях
-- и по ключевым словам в навыках
where (name ilike '%data scientist%' 
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name ilike '%ML%'
    and name not ilike '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and (key_skills ilike '%python%')'''

In [111]:
# результат запроса
python_ds_vac_df = pd.read_sql_query(python_ds_vac, connection)
python_ds_vac_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\1261125289.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  python_ds_vac_df = pd.read_sql_query(python_ds_vac, connection)


,count_ds_vac_with_py_skills
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [114]:
# текст запроса
avg_ds_key_skills = f'''select
-- выводим среднее количество ключевых навыков
    avg(length(key_skills)-length(replace(key_skills, CHR(9), ''))+1) AS average_of_key_skills
-- обращаемся к таблице вакансий
from vacancies 
-- фильтруем по ключевым словам в вакансиях
where (name ilike '%data scientist%'
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name like '%ML%' 
    and name not ilike '%HTML%') 
    or name ilike '%machine learning%' 
    or name ilike '%машинн%обучен%')'''

In [115]:
# результат запроса
avg_ds_key_skills_df = pd.read_sql_query(avg_ds_key_skills, connection)
round(avg_ds_key_skills_df, 2)

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\4019775196.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_ds_key_skills_df = pd.read_sql_query(avg_ds_key_skills, connection)


,average_of_key_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [116]:
# текст запроса
salary_ds = f'''select
-- выводим опыт работы
-- и среднюю заработную плату 
    experience,
    avg(coalesce((salary_from + salary_to) / 2, salary_from, salary_to)) as average_salary 
-- обращаемся к таблице вакансий
from vacancies
-- фильтруем по ключевым словам в вакансиях,
-- по опыту работы
--  и по непустым столбцам с указанием зарплаты 
where (name ilike '%data scientist%'
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name like '%ML%' 
    and name not ilike '%HTML%') 
    or name ilike '%machine learning%' 
    or name ilike '%машинн%обучен%')
    and (experience = 'Более 6 лет'
    or experience = 'От 3 до 6 лет'
    or experience = 'От 1 года до 3 лет'
    or experience = 'Нет опыта')
    and (salary_from is not null or salary_to is not null)
group by experience 
order by experience desc'''

In [117]:
# результат запроса
salary_ds_df = pd.read_sql_query(salary_ds, connection)
round(salary_ds_df)

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\3803636989.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_ds_df = pd.read_sql_query(salary_ds, connection)


,experience,average_salary
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

# Выводы по предметному анализу

В финальном блоке проекта был проведен предметный анализ вакансий, в частности, поскольку интерес проекта сосредоточен на вакансиях в сфере IT, бы проведен анализ вакансий на позицию Data scientist (далее - DS).

Так, согласно проведенному анализу общее количество вакансий имеющих отношения к данным - 1 771. Количество вакансий для DS уровня junior (начинающий специалист), согласно критериям выполняемого задания - 51. Среднее количество навыков необходимых DS - 6.41, при этом в качестве ключевых навыков в 357 вакансиях указаны знания и умения программировать на языке Python и в 220 вакансиях требуется знание написания sql - запросов.

Также примечателен тот факт, что для специалистов DS высокого уровня с опытом работы более 6 лет не указанна заработная плата. Из этого можно сделать вывод, что для специалистов с таким опытом работы заработная плата обсуждается индивидуально. Для специалистов начального уровня средняя зарплата составляет 74 636 рублей, что является выше среднего уровня нижнего порога зарплатной вилки в целом на рынке труда.

# Общий вывод по проекту

# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

Подводя итоги исследования можно сделать несколько выводов:
1. Рынок труда полон вакансий;
2. Рынок труда имеет обширную географию;
3. Основополагающее большинство вакансий сосредоточенно в крупных городах;
4. Среднее значение нижнего порога зарплатной вилки выше среднего уровня зарплаты по России.

Что качается IT индустрии, в частности DS, можно сделать вывод, что специалист настоящей профессии имеет довольно высокую оплату труда, а также довольно большое количество ключевых навыков, требующихся для качественного выполнения трудовых обязанностей.

# Дополнительные исследования данных

1. Сколько есть подходящих вакансий для среднего уровня специалиста в data science

In [118]:
mds_vac = f'''select
-- выводим количество вакансий
    count(name) AS count_middle_ds_vac
-- обращаемся к таблице вакансий
from vacancies
-- фильтруем по ключевым словам в вакансиях
-- и по ключемым словам в опыте работы и типе занятости
where (name ilike '%data scientist%' 
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name ilike '%ML%'
    and name not ilike '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and 
    (name ilike '%middle%'
    or experience = 'От 1 года до 3 лет'
    or employment ilike '%Проектная работа%'
    or employment ilike '%Полная занятость%')'''

In [119]:
mds_vac_df = pd.read_sql_query(mds_vac, connection)
mds_vac_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\1620533634.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mds_vac_df = pd.read_sql_query(mds_vac, connection)


,count_middle_ds_vac
0,525


2. Сколько есть подходящих вакансий для специалиста высокого уровня в data science

In [120]:
s_or_tl_ds_vac = f'''select
-- выводим количество вакансий
    count(name) AS count_senior_or_team_lead_ds_vac
-- обращаемся к таблице вакансий
from vacancies
-- фильтруем по ключевым словам в вакансиях
-- и по ключемым словам в опыте работы и типе занятости
where (name ilike '%data scientist%' 
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or (name ilike '%ML%'
    and name not ilike '%HTML%')
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and 
    (name ilike '%senior%'
    or name ilike '%team lead%'
    or experience = 'От 3 до 6 лет'
    or experience = 'Более 6 лет'
    or employment ilike '%Проектная работа%'
    or employment ilike '%Полная занятость%')'''

In [121]:
s_or_tl_ds_vac_df = pd.read_sql_query(s_or_tl_ds_vac, connection)
s_or_tl_ds_vac_df

C:\Users\IAMMI\AppData\Local\Temp\ipykernel_6888\1143526534.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s_or_tl_ds_vac_df = pd.read_sql_query(s_or_tl_ds_vac, connection)


,count_senior_or_team_lead_ds_vac
0,527


***

В качестве продолжения исследования было проанализированно колличество ванкансий представленных на рынке труда для специалистов DS среднего и высокого уровней. Так, согласно анализу, можно сделать вывод, что рынок труда больше заинтересован в специалистах настоящей профессии имеющих средний и высокий уровень квалификации, нежеле чем в специалистах начального уровня.  